In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings("ignore")

In [18]:
data = pd.read_csv("VIX_YF_data.csv", parse_dates=["Date"])
data.set_index("Date", inplace=True)

# Drop any rows with missing values
data = data.dropna()

# Scale the features (excluding VIX)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.drop(columns=["VIX"]))

# Separate target variable (VIX) for later evaluation, without scaling
vix = data["VIX"].values


In [19]:
import numpy as np

sequence_length = 60
X_lstm, y_lstm = [], []
for i in range(sequence_length, len(scaled_data)):
    X_lstm.append(scaled_data[i-sequence_length:i])
    y_lstm.append(vix[i])  # Use unscaled VIX values as the target

X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

In [20]:
split = int(0.8 * len(X_lstm))
X_train, y_train = X_lstm[:split], y_lstm[:split]
X_test, y_test = X_lstm[split:], y_lstm[split:]

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)  # Predicting VIX
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
predictions = model.predict(X_test)

mse_value = np.mean((y_test - predictions) ** 2)
print("MSE:", mse_value)